In [4]:
# Go to https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/0TJX8Y

# Unzip the file named 'dataverse_files' to a directory of your choice on your hard drive.
# [This may take a few minutes.]
# Untar UNGDC_1946-2023.tgz

# In the directory you have chosen, you now have more than 8,000 speeches in plain text format (with UTF-8 encoding).
# Each speech is named using the following convention: ISO 3166-1 alpha-3 country code, followed by the UN Session number, followed by year.
# E.g. USA_73_2018.txt is the full text of the speech the United States' representative gave during the UN's 73rd General Debate session in 2018.

# In the dir there's a list of folder each containing the speeches of a session. So `Session 01 - 1946` contains the speeches of the first session in 1946.
dir = "dataverse_files/UN General Debate Corpus/TXT"


In [5]:
# We're going to loop over all sessions and store the speech country (ISO code),
# session number, year and text content in a sqlite3 database.
# Schema of the database:
# country: ISO 3166-1 alpha-3 country code
# session: UN session number
# year: year of the speech
# text: full text of the speech
# The combination of country, session and year should be unique.

import sqlite3

conn = sqlite3.connect("un_speeches.db")
c = conn.cursor()

c.execute("""CREATE TABLE IF NOT EXISTS speeches (
    country TEXT,
    session INTEGER,
    year INTEGER,
    text TEXT,
    PRIMARY KEY (country, session, year)
)""")
conn.commit()

In [6]:
# Go over all the folders in the dir and load the speech data in the database
# Get ISO code, session number and year from the filename, not folder
# Filename looks like this: USA_73_2018.txt

import os

for folder in os.listdir(dir):
    folder_path = os.path.join(dir, folder)

    if not os.path.isdir(folder_path):
        continue

    for filename in os.listdir(folder_path):
        if not filename.endswith(".txt"):
            continue

        # First validate the the filename adheres to schema ISO_xx_xxxx.txt
        if len(filename.split("_")) != 3:
            print(f"Skipping file {filename} as it does not adhere to the schema.")
            continue

        country, session, year = filename.replace(".txt", "").split("_")
        with open(os.path.join(folder_path, filename), "r") as f:
            text = f.read()

        # Check if the entry already exists
        c.execute(
            "SELECT 1 FROM speeches WHERE country = ? AND session = ? AND year = ?",
            (country, session, year),
        )
        if c.fetchone() is None:
            print(f"Inserting {country}, {session}, {year}")
            c.execute(
                "INSERT INTO speeches VALUES (?, ?, ?, ?)",
                (country, session, year, text),
            )
            conn.commit()
        else:
            print(
                f"Entry for {country}, {session}, {year} already exists. Skipping insertion."
            )

conn.close()
print("Done.")

Entry for BEL, 05, 1950 already exists. Skipping insertion.
Entry for BLR, 05, 1950 already exists. Skipping insertion.
Entry for FRA, 05, 1950 already exists. Skipping insertion.
Entry for PAK, 05, 1950 already exists. Skipping insertion.
Entry for TUR, 05, 1950 already exists. Skipping insertion.
Entry for BRA, 05, 1950 already exists. Skipping insertion.
Entry for ETH, 05, 1950 already exists. Skipping insertion.
Entry for DOM, 05, 1950 already exists. Skipping insertion.
Entry for URY, 05, 1950 already exists. Skipping insertion.
Entry for PAN, 05, 1950 already exists. Skipping insertion.
Entry for VEN, 05, 1950 already exists. Skipping insertion.
Entry for YUG, 05, 1950 already exists. Skipping insertion.
Entry for GRC, 05, 1950 already exists. Skipping insertion.
Entry for PHL, 05, 1950 already exists. Skipping insertion.
Entry for POL, 05, 1950 already exists. Skipping insertion.
Entry for CHN, 05, 1950 already exists. Skipping insertion.
Entry for ECU, 05, 1950 already exists. 

In [23]:
# Next step is adding the additional data about the speeches that is stored in an Excel file.

excel_file = "dataverse_files/UN General Debate Corpus/Speakers_by_session.xlsx"

# The Excel file contains the following columns:
# Year;Session;ISO Code;Country;Name of Person Speaking;Post;

# We're going to add the name of the person speaking and their post to the database.
# Schema of the database:
# country: ISO 3166-1 alpha-3 country code
# session: UN session number
# year: year of the speech
# text: full text of the speech
# country_name: name of the country
# speaker: name of the person speaking
# post: post of the speaker


conn = sqlite3.connect("un_speeches.db")
c = conn.cursor()


# Add the columns to the database, if they don't exist
def add_column_if_not_exists(cursor, table_name, column_name, column_type):
    cursor.execute(f"PRAGMA table_info({table_name})")
    cols = [info[1] for info in cursor.fetchall()]
    if column_name not in cols:
        cursor.execute(
            f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_type}"
        )


add_column_if_not_exists(c, "speeches", "country_name", "TEXT")
add_column_if_not_exists(c, "speeches", "speaker", "TEXT")
add_column_if_not_exists(c, "speeches", "post", "TEXT")


conn.commit()


In [26]:
# Load the data from the Excel file and add it to the database
import openpyxl

wb = openpyxl.load_workbook(excel_file)
sheet = wb.active

conn = sqlite3.connect("un_speeches.db")
c = conn.cursor()

for row in sheet.iter_rows(min_row=2, values_only=True):
    year, session, country, country_name, speaker, post, *_ = row

    # Check if the entry already exists
    c.execute(
        "SELECT 1 FROM speeches WHERE country = ? AND session = ? AND year = ?",
        (country, session, year),
    )
    if c.fetchone() is None:
        print(
            f"Skipping {country}, {session}, {year} as it does not exist in the speeches table."
        )
        continue

    print(
        f"Updating {country}, {session}, {year}. Adding {country_name}, {speaker}, {post}"
    )
    c.execute(
        "UPDATE speeches SET country_name = ?, speaker = ?, post = ? WHERE country = ? AND session = ? AND year = ?",
        (country_name, speaker, post, country, session, year),
    )
    conn.commit()

conn.close()
print("Done.")

Updating BRA, 78, 2023. Adding Brazil , Luiz Inacio Lula da Silva, President
Updating USA, 78, 2023. Adding United States of America, Joseph R. Biden, President
Updating COL, 78, 2023. Adding Colombia, Gustavo Petro Urrego, President
Updating JOR, 78, 2023. Adding Jordan, Abdullah II ibn Al Hussein, King
Updating POL, 78, 2023. Adding Poland, Andrzej Duda, President
Updating CUB, 78, 2023. Adding Cuba, Miguel Diaz-Canel Bermudez, President
Updating TUR, 78, 2023. Adding Turkey, Recep Tayyip Erdogan, President
Updating PRT, 78, 2023. Adding Portugal , Marcelo Rebelo de Sousa, President
Updating QAT, 78, 2023. Adding Qatar, Tamim bin Hamad Al Thani, Amir
Updating ZAF, 78, 2023. Adding South Africa, Matamela Cyril Ramaphosa, President
Updating TKM, 78, 2023. Adding Turkmenistan , Serdar Berdimuhamedov, President
Updating UKR, 78, 2023. Adding Ukraine , Volodymyr Zelenskyy, President
Updating GTM, 78, 2023. Adding Guatemala, Alejandro Giammattei Falla, President
Updating HUN, 78, 2023. Add

In [36]:
# Do a count of all speeches made by kings for every year

conn = sqlite3.connect("un_speeches.db")
c = conn.cursor()

c.execute(
    """
    SELECT year, COUNT(*) as count
    FROM speeches
    WHERE post LIKE '%king%'
       OR post LIKE '%queen%'
       OR post LIKE '%prince%'
       OR post LIKE '%princess%'
    GROUP BY year
    ORDER BY year
    """
)

for row in c.fetchall():
    print(row)


(1954, 1)
(1960, 3)
(1963, 1)
(1979, 1)
(1981, 2)
(1985, 1)
(1986, 2)
(1987, 1)
(1991, 2)
(1992, 1)
(1993, 1)
(2015, 2)
(2016, 3)
(2017, 4)
(2018, 2)
(2019, 4)
(2020, 4)
(2021, 3)
(2022, 3)
(2023, 3)


In [ ]:
# Show all the names of the kings
c.execute(
    """
    SELECT DISTINCT speaker, year, country_name
    FROM speeches
    WHERE post LIKE '%king%'
       OR post LIKE '%queen%'
       OR post LIKE '%prince%'
       OR post LIKE '%princess%'
    ORDER BY year
    """
)

for row in c.fetchall():
    print(row)

('Prince WAN WAITHAYAKON', 1954, 'Thailand')
('Prince Norodom SIHANOUK ', 1960, 'Cambodia')
('Prince Moulay HASSAN', 1960, 'Morocco')
('H. M. HUSSEIN ', 1960, 'Jordan')
('Souvanna Phovma', 1963, 'Laos')
('King Hussein', 1979, 'Jordan')
('Crown Prince HASSAN', 1981, 'Jordan')
('Prince ALFAISAL', 1981, 'Saudi Arabia')
('King Hussein', 1985, 'Jordan')
('King Juan Carlos', 1986, 'Spain')
('Prince AL-FAISAL', 1986, 'Saudi Arabia')
('KING MOSHOESHOE II ', 1987, 'Lesotho')
('Prince Hans-Adam II ', 1991, 'Liechtenstein')
('King Mswati III ', 1991, 'Swaziland')
('Mnhamed BOLKIAH', 1992, 'Brunei Darussalam')
('Prince Albert', 1993, 'Monaco')
('King Abdullah II ibn Al Hussein', 2015, 'Jordan')
('Willem-Alexander', 2015, 'Netherlands')
('King Abdullah II ibn Al Hussein', 2016, 'Jordan')
('Prince Mohammed bin Naif bin Abdulaziz Al-Saud', 2016, 'Saudi Arabia')
('King Don Felipe VI', 2016, 'Spain')
('King Tupou VI', 2017, 'Tonga')
('Prince Albert II', 2017, 'Monaco')
('Crown Prince Al Hussein bin Abd

In [ ]:
# Get a list of all distinct possible posts
c.execute(
    """
    SELECT DISTINCT post
    FROM speeches
    ORDER BY post
    """
)

for row in c.fetchall():
    print(row)

(None,)
(' ',)
(' Coordinator of the Junta of the Government',)
(' Deputy Chairman of the Cabinet of Ministers and Minister for Foreign Affairs',)
('Acting Head of State ',)
('Acting Minister for Foreign Affairs and International Cooperation',)
('Acting President',)
('Amir',)
('Attorney General and Minister for Foreign Affairs',)
('Attorney General and Minister for Foreign Affairs and Foreign Trade',)
('Attourney General and Minister for Foreign Affairs and Foreign Trade',)
('CHAIRMAN OF THE SUPREME COUNCIL',)
('CROWN PRINCE',)
('Cabinet Secretary for Foreign Affairs and International Trade',)
('Cairman of the Presidency',)
('Chair of Delegation',)
('Chair of the Delegation',)
('Chair of the delegation',)
('Chairman',)
('Chairman of the Assembly Presidium',)
('Chairman of the Council of Ministers',)
("Chairman of the Council of Ministers of the People's Republic of Albania",)
('Chairman of the Council of Ministers of the Union of Soviet Socialist Republics',)
('Chairman of the Presiden